In [1]:
!nvidia-smi

Tue Jun  4 23:38:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:3B:00.0 Off |                    0 |
| N/A   20C    P0              26W / 165W |    252MiB / 24576MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
# os.environ["WANDB_SILENT"] = "true"

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
print(os.environ["CUDA_VISIBLE_DEVICES"])

1


# Go to Table of Contents

1. [Go to TOC](#gototoc)

In [3]:
def getsplit(lang_split_idx, key='train_idx'):
    idx_list = [v[key] for k,v in lang_split_idx.items()]
    # print(len(idx_list))
    idx_list = [i for eachlist in idx_list for i in eachlist]
    return idx_list

In [4]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [5]:
revised = True
newsplit = True
filename = '_revisedcateg' if revised else ''
folder = 'final_' if newsplit else ''

ROOT = Path('/gaueko0/users/nmishra/multiling_fludetection')
DATA_PATH = ROOT.joinpath('data')
PARAMS_FILE = ROOT.joinpath('params_revisedcateg.tsv')

OUT_PATH = ROOT.joinpath(f'{folder}evals/eval_revisedcateg')
OUT_PATH

PosixPath('/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg')

In [6]:
# load data
all_data = pd.read_csv(DATA_PATH.joinpath(f'all/alldata{filename}.csv'))
print(all_data.shape)

full_data = pd.read_csv(DATA_PATH.joinpath(f'all/fulldata{filename}.csv'))
full_data = full_data.convert_dtypes()

print(full_data.shape)
full_data.head()

(4284, 6)
(21420, 9)


,id,tweet,target_lang,lang,year,final_annotation,overlap,eval_lang,original
0,21850866789130240,"Hola Twitter. En 2010 casi me muero de gripe, ...",es,fr,2011,1. Likely ILI infection,0,fr-es,False
1,21850866789130240,Hallo Twitter. 2010 bin ich fast an einer Grip...,de,fr,2011,1. Likely ILI infection,0,fr-de,False
2,21850866789130240,"Hello Twitter. In 2010, I almost died of the f...",en,fr,2011,1. Likely ILI infection,0,fr-en,False
3,21850866789130240,Ciao Twitter. Nel 2010 ho rischiato di morire ...,it,fr,2011,1. Likely ILI infection,0,fr-it,False
4,21850866789130240,"Bonjour Twitter. En 2010, j'ai frôlé la mort p...",fr,fr,2011,1. Likely ILI infection,0,fr-fr,True


In [7]:
# load splits for all data
params = pd.read_csv(PARAMS_FILE, sep='\t')
SPLITS = params['split'].unique()
SPLITS

array(['0.6,0.2,0.2'], dtype=object)

## A. Get the ids associated with each split per language

In [8]:
# for split in SPLITS:
split = SPLITS[0]
print(split)

# read data split index
dirname = f"testset{split.replace(',','_')}"
split_path = OUT_PATH.joinpath(dirname)
print(f"Reading data split index from: {split_path}")
with open(split_path.joinpath('split_idx.json'), 'r') as f:
    split_idx = json.load(f) 

# get associated ids
split_id = {}
for lang, splits_dict in split_idx.items():
    lang_split_id = {}
    print(lang)
    for split_type, idx in splits_dict.items():
        print(f'  {split_type}')
        sub_df = all_data.iloc[idx]['id'].values
        lang_split_id[split_type] = all_data.iloc[idx]['id'].values
    split_id[lang] = lang_split_id

0.6,0.2,0.2
Reading data split index from: /gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2
de
  train_idx
  test_idx
  valid_idx
en
  train_idx
  test_idx
  valid_idx
es
  train_idx
  test_idx
  valid_idx
fr
  train_idx
  test_idx
  valid_idx
it
  train_idx
  test_idx
  valid_idx


In [9]:
# # check if any index is duplicated
# test_idx_df = pd.DataFrame.from_dict(split_idx)
# for col in test_idx_df.columns:
#     print(col, test_idx_df[[col]].explode(col).duplicated().sum())

In [10]:
# # check each index is uniquely placed in one split type of one language only
# for i in range(21):
#     n = np.random.randint(0, all_data.shape[0])
#     print(n)
#     test_idx_df = pd.DataFrame.from_dict(split_idx)
#     test_idx_df = test_idx_df[test_idx_df.map(lambda x: n in x)]
#     if (test_idx_df.notna().sum().sum()==1) or test_idx_df.notna().sum().sum()==0:
#         print('matched only once or not at all')
#     else:
#         print('matched more than once')

In [11]:
# # check if any id is duplicated
# test_id_df = pd.DataFrame.from_dict(split_id)
# for col in test_id_df.columns:
#     print(col, test_id_df[[col]].explode(col).duplicated().sum())

In [12]:
# # check each id is uniquely placed in one split type of one language only
# for i in range(21):
#     n = np.random.randint(0, all_data.shape[0])
#     print(n)
#     test_id_df = pd.DataFrame.from_dict(split_id)
#     test_id_df = test_id_df[test_id_df.map(lambda x: n in x)]
#     if (test_id_df.notna().sum().sum()==1) or test_id_df.notna().sum().sum()==0:
#         print('matched only once or not at all')
#     else:
#         print('matched more than once')

In [13]:
train_ids = getsplit(split_id, key='train_idx')
valid_ids = getsplit(split_id, key='valid_idx')
test_ids  = getsplit(split_id, key='test_idx')
print(f"Distribution of data in train, validation and test splits: {len(train_ids)}, {len(valid_ids)}, {len(test_ids)}")

Distribution of data in train, validation and test splits: 2567, 857, 860


In [14]:
test_data = all_data[all_data['id'].isin(test_ids)]
test_data.groupby('lang')['final_annotation'].value_counts()

lang  final_annotation                           
de    3. Not Related to ILI or COVID-19 Infection     99
      1. Likely ILI infection                         95
en    3. Not Related to ILI or COVID-19 Infection     22
      1. Likely ILI infection                         18
es    3. Not Related to ILI or COVID-19 Infection    120
      1. Likely ILI infection                        113
fr    3. Not Related to ILI or COVID-19 Infection    108
      1. Likely ILI infection                         85
it    3. Not Related to ILI or COVID-19 Infection    170
      1. Likely ILI infection                         30
Name: count, dtype: int64

# Table of contents
<a name="gototoc"></a>
1. [Original](#original)
2. [Learning Curve](#learning_curve)
3. [One to Many](#onetomany)
4. [Many to One](#manytoone)
5. [Train Original Test Many to One](#trainoriginaltestmanytoone)

<a name="original"></a>

## 1 Original
### i. Get split index for each split of each language

In [15]:
experiment_type = 'original'

In [16]:
translate_test = True
original_split_idx = {}

for lang, splits_dict in split_id.items():
    original_lang_split_idx = {}
    
    for split_type, ids in splits_dict.items():                           # for each split in each language
        
        if 'test' in split_type:                                          # if test
            sub_df = full_data[full_data['id'].isin(test_ids)]            # take all test ids
            # sub_df = sub_df[sub_df['lang']==sub_df['target_lang']]        # take only original
        
        else:                                                             # otherwise
            sub_df = full_data[full_data['id'].isin(ids)]                 # take split type ids for the language in loop only
            sub_df = sub_df[sub_df['lang']==sub_df['target_lang']]        # take only original
        print(lang, split_type, sub_df.shape)
        original_lang_split_idx[split_type] = sub_df.index.tolist()       # save the index of the selected ids
        
    original_split_idx[lang] = original_lang_split_idx
    print('-'*13)

print(original_split_idx.keys())

de train_idx (579, 9)
de test_idx (4300, 9)
de valid_idx (194, 9)
-------------
en train_idx (117, 9)
en test_idx (4300, 9)
en valid_idx (39, 9)
-------------
es train_idx (698, 9)
es test_idx (4300, 9)
es valid_idx (233, 9)
-------------
fr train_idx (576, 9)
fr test_idx (4300, 9)
fr valid_idx (192, 9)
-------------
it train_idx (597, 9)
it test_idx (4300, 9)
it valid_idx (199, 9)
-------------
dict_keys(['de', 'en', 'es', 'fr', 'it'])


In [17]:
# check each index is uniquely placed in one split type of one language only
# test_set will have repeat here 
all_split_types = original_split_idx.get('de').keys()
print(all_split_types)

for type_to_test in all_split_types:
    for i in range(21):
        n = np.random.randint(0, full_data.shape[0])
        test_df = pd.DataFrame.from_dict(original_split_idx)
        test_df = test_df[test_df.index==type_to_test]
        test_df = test_df[test_df.map(lambda x: n in x)]
        if (test_df.notna().sum().sum()==1) or test_df.notna().sum().sum()==0:
            pass
        else:
            print(type_to_test)
            print(n)
            print('matched more than once')

dict_keys(['train_idx', 'test_idx', 'valid_idx'])
test_idx
5943
matched more than once
test_idx
16173
matched more than once
test_idx
7306
matched more than once
test_idx
18516
matched more than once


In [18]:
split_type = 'test_idx'
split_type = 'train_idx'
split_type = 'valid_idx'
language = 'it'
full_data[full_data.index.isin(original_split_idx[language][split_type])]

,id,tweet,target_lang,lang,year,final_annotation,overlap,eval_lang,original
3106,1043436583578423296,L 'AMORE NON È POSSESSO. Io sono ogni giorno ...,it,it,2018,3. Not Related to ILI or COVID-19 Infection,1,it-it,True
3309,1045676821977665541,"#LaPalabraDelDía di oggi è ""almena"", questi sp...",it,it,2018,3. Not Related to ILI or COVID-19 Infection,1,it-it,True
3440,1047834383867158529,"Il mio prof mi ha accattato l'influenza, GRAZI...",it,it,2018,1. Likely ILI infection,0,it-it,True
3971,1059752080095744005,"@user Tranquillo, appena realizzi che non può ...",it,it,2018,3. Not Related to ILI or COVID-19 Infection,1,it-it,True
4023,1060245545824989184,buona cena. stasera brodino vegetale e patale ...,it,it,2018,1. Likely ILI infection,1,it-it,True
...,...,...,...,...,...,...,...,...,...
21053,1630860808329494528,@user @user @user C'entra molto più della tua ...,it,it,2023,3. Not Related to ILI or COVID-19 Infection,0,it-it,True
21112,1631635285422268418,Questo nuovo ordine mondiale multipolare è car...,it,it,2023,3. Not Related to ILI or COVID-19 Infection,0,it-it,True
21232,1633444670914146304,@user OK. San Marino e Vaticano sono nella zon...,it,it,2023,3. Not Related to ILI or COVID-19 Infection,0,it-it,True
21370,1635581813409161216,@user @user @user uno dei problemi è capire (e...,it,it,2023,3. Not Related to ILI or COVID-19 Infection,0,it-it,True


### ii. Get split index for each split for all languages

In [21]:
original_lang_split_idx = {}

original_df = full_data[full_data['lang']==full_data['target_lang']]
translated_df = full_data[full_data['lang']!=full_data['target_lang']]

original_lang_split_idx['train_idx'] = original_df[original_df['id'].isin(train_ids)].index.tolist()                # take all test ids
original_lang_split_idx['test_idx']  = full_data[full_data['id'].isin(test_ids)].index.tolist()
original_lang_split_idx['valid_idx'] = original_df[original_df['id'].isin(valid_ids)].index.tolist()
print([len(i) for i in original_lang_split_idx.values()])

original_split_idx['all'] = original_lang_split_idx
print(original_split_idx.keys())

[2567, 4300, 857]
dict_keys(['de', 'en', 'es', 'fr', 'it', 'all'])


In [22]:
# # save the split index
print(split_path.joinpath(f'split_idx_{experiment_type}.json'))
with open(split_path.joinpath(f'split_idx_{experiment_type}.json'), 'w') as f:
    json.dump(original_split_idx, f)

/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/split_idx_original.json


<a name="learning_curve"></a>

## 2 Learning Curve
### Get split index for one language for each fraction/partition of data

In [9]:
language = 'es'
experiment_type = f'original_{language}_learningcurve'

In [20]:
fractions = [0.25, 0.50, .70]
fractions_name = ['one', 'two', 'three']
n_splits = len(fractions)
lang_split_idx = split_idx[language]
fraction_idx_len = {}
for i in range(n_splits):
    fraction_idx_len[f'{language}_{fractions_name[i]}'] = {split_type: int( len( lang_split_idx[split_type] )*fractions[i] ) 
                                                           for split_type in lang_split_idx}
print(fraction_idx_len)

{'es_one': {'train_idx': 174, 'test_idx': 58, 'valid_idx': 58}, 'es_two': {'train_idx': 349, 'test_idx': 116, 'valid_idx': 116}, 'es_three': {'train_idx': 488, 'test_idx': 163, 'valid_idx': 163}}


In [22]:
fractional_split_idx = {}
for each_split in fraction_idx_len:
    print(each_split)
    idx_len = fraction_idx_len[each_split]
    each_split_idx = {}
    for split_type, split_type_ids in split_id[language].items():
        if 'test' in split_type:
            each_split_idx[split_type] = full_data[full_data['id'].isin(test_ids)].index.tolist()  # take all test ids
        else:
            sub_df = full_data[full_data['id'].isin(split_type_ids)]                         # take the associated ids in full data
            each_split_ids = sub_df[sub_df['target_lang']==sub_df['lang']].index.tolist()    # take index of original items only
            each_split_idx[split_type] = each_split_ids[ :idx_len[split_type]]               # get fraction of idx
        fractional_split_idx[each_split] = each_split_idx
        print(split_type, len(each_split_idx[split_type]))

es_one
train_idx 174
test_idx 4300
valid_idx 58
es_two
train_idx 349
test_idx 4300
valid_idx 116
es_three
train_idx 488
test_idx 4300
valid_idx 163


In [25]:
full_data[full_data.index.isin(fractional_split_idx.get('es_three')['valid_idx'])]

,id,tweet,target_lang,lang,year,final_annotation,overlap,eval_lang
3049,1041876687988948993,Yo sé que las mujeres tienen muchísimas aptitu...,es,es,2018,1. Likely ILI infection,0,es-es
3160,1044350928227364869,Hace como 1 mes que estoy con esta gripe del o...,es,es,2018,1. Likely ILI infection,1,es-es
3188,1044540627491475456,"Que mañana vaya alguien de mi parte a currar, ...",es,es,2018,1. Likely ILI infection,0,es-es
3266,1045317807263617024,Tengo una gripe...,es,es,2018,1. Likely ILI infection,1,es-es
3333,1045966224465768448,"Mis tres gatos están con fiebre, mi madre y yo...",es,es,2018,1. Likely ILI infection,0,es-es
...,...,...,...,...,...,...,...,...
16468,1473039392809885697,@user @user Eso lo entiendo (aunque siendo el ...,es,es,2021,3. Not Related to ILI or COVID-19 Infection,0,es-es
16480,1473529806184755205,@user Lo siento ami Bella . Pendiente con lo ...,es,es,2021,1. Likely ILI infection,1,es-es
16516,1473725280485318662,@user Si estoy de acuerdo en lo de la atención...,es,es,2021,3. Not Related to ILI or COVID-19 Infection,0,es-es
16536,1473947371159662592,"@user Hemos acabado con la gripe, ha sido uno ...",es,es,2021,3. Not Related to ILI or COVID-19 Infection,0,es-es


In [26]:
fractional_split_idx.keys()

dict_keys(['es_one', 'es_two', 'es_three'])

In [27]:
fractional_split_idx.get('es_one').keys()

dict_keys(['train_idx', 'test_idx', 'valid_idx'])

In [29]:
# # save the split index
print(split_path.joinpath(f'split_idx_{language}_learningcurve.json'))
# with open(split_path.joinpath(f'split_idx_{language}_learningcurve.json'), 'w') as f:
#     json.dump(fractional_split_idx, f)

/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/split_idx_es_learningcurve.json


In [27]:
# # ## 2 Original Translate Test
# experiment_type = 'original_ttest'
# translate_test = True
# original_split_idx = {}

# for lang, splits_dict in split_id.items():
#     original_lang_split_idx = {}
    
#     for split_type, ids in splits_dict.items():                           # for each split in each language
        
#         if 'test' in split_type:                                          # if test
#             sub_df = full_data[full_data['id'].isin(test_ids)]                # take all test ids
#             sub_df = sub_df[sub_df['target_lang']==lang]                      # take only translations to that language
#             sub_df = sub_df[sub_df['lang']!=sub_df['target_lang']]            # remove original as already added in test
        
#         else:                                                              # otherwise
#             sub_df = full_data[full_data['id'].isin(ids)]                  # take split type ids for the language in loop only
#             sub_df = sub_df[sub_df['lang']==sub_df['target_lang']]         # take only original
        
#         print(lang, split_type, sub_df.shape)
#         original_lang_split_idx[split_type] = sub_df.index.tolist()        # save the index of the selected ids
        
#     original_split_idx[lang] = original_lang_split_idx
#     print('-'*13)

# print(original_split_idx.keys())

In [28]:
# # check each index is uniquely placed in one split type of one language only
# # test_set will have repeat here 
# all_split_types = original_split_idx.get('de').keys()
# print(all_split_types)

# for type_to_test in all_split_types:
#     for i in range(21):
#         n = np.random.randint(0, full_data.shape[0])
#         test_df = pd.DataFrame.from_dict(original_split_idx)
#         test_df = test_df[test_df.index==type_to_test]
#         test_df = test_df[test_df.map(lambda x: n in x)]
#         if (test_df.notna().sum().sum()==1) or test_df.notna().sum().sum()==0:
#             pass
#         else:
#             print(type_to_test)
#             print(n)
#             print('matched more than once')

In [29]:
# original_lang_split_idx = {}

# original_df = full_data[full_data['lang']==full_data['target_lang']]
# translated_df = full_data[full_data['lang']!=full_data['target_lang']]

# original_lang_split_idx['train_idx'] = original_df[original_df['id'].isin(train_ids)].index.tolist()                # take all test ids
# original_lang_split_idx['test_idx']  = translated_df[translated_df['id'].isin(test_ids)].index.tolist() 
# original_lang_split_idx['valid_idx'] = original_df[original_df['id'].isin(valid_ids)].index.tolist()
# print([len(i) for i in original_lang_split_idx.values()])

# original_split_idx['all'] = original_lang_split_idx
# print(original_split_idx.keys())

In [30]:
# full_data[full_data.index.isin(original_split_idx['all']['test_idx'])]

In [31]:
# # save the split index
# print(split_path.joinpath(f'split_idx_{experiment_type}.json'))
# with open(split_path.joinpath(f'split_idx_{experiment_type}.json'), 'w') as f:
#     json.dump(original_split_idx, f)

<a name="onetomany"></a>

## 3 One to Many

### Get split index for each split of each language

In [23]:
experiment_type = 'one_to_many'

In [24]:
onetomany_split_idx = {}
for lang, splits_dict in split_id.items():
    onetomany_lang_split_idx = {}
    for split_type, ids in splits_dict.items():                           # for each split in each language
        if 'test' in split_type:
            sub_df = full_data[full_data['id'].isin(test_ids)]                # take all test ids
            # sub_df = sub_df[sub_df['target_lang']==sub_df['lang']]            # take originals only
            # onetomany_lang_split_idx[split_type] = sub_df.index.tolist()    # test on multilingual original 
            # print(lang, split_type, f'  {len(sub_df)}')
        else:
            sub_df = full_data[full_data['id'].isin(ids)]                     # take only id in the split_type for that langauge
        print(lang, split_type, f'  {sub_df.shape[0]}')
        # print(sub_df[['id','lang']].head())
        onetomany_lang_split_idx[split_type] = sub_df.index.tolist() # get the index of the selected ids and save into respective split
    onetomany_split_idx[lang] = onetomany_lang_split_idx

de train_idx   2895
de test_idx   4300
de valid_idx   970
en train_idx   585
en test_idx   4300
en valid_idx   195
es train_idx   3490
es test_idx   4300
es valid_idx   1165
fr train_idx   2880
fr test_idx   4300
fr valid_idx   960
it train_idx   2985
it test_idx   4300
it valid_idx   995


In [25]:
# check each index is uniquely placed in one split type of one language only
# test_set will have repeat here 
all_split_types = onetomany_split_idx.get('de').keys()
print(all_split_types)

for type_to_test in all_split_types:
    for i in range(21):
        n = np.random.randint(0, full_data.shape[0])
        test_df = pd.DataFrame.from_dict(onetomany_split_idx)
        test_df = test_df[test_df.index==type_to_test]
        test_df = test_df[test_df.map(lambda x: n in x)]
        if (test_df.notna().sum().sum()==1) or test_df.notna().sum().sum()==0:
            pass
        else:
            print(type_to_test)
            print(n)
            print('matched more than once')

dict_keys(['train_idx', 'test_idx', 'valid_idx'])
test_idx
7500
matched more than once
test_idx
18310
matched more than once
test_idx
13874
matched more than once
test_idx
9683
matched more than once


In [26]:
split_type = 'test_idx'
split_type = 'train_idx'
split_type = 'valid_idx'
language = 'fr'
full_data[full_data.index.isin(onetomany_split_idx[language][split_type])]

,id,tweet,target_lang,lang,year,final_annotation,overlap,eval_lang,original
0,21850866789130240,"Hola Twitter. En 2010 casi me muero de gripe, ...",es,fr,2011,1. Likely ILI infection,0,fr-es,False
1,21850866789130240,Hallo Twitter. 2010 bin ich fast an einer Grip...,de,fr,2011,1. Likely ILI infection,0,fr-de,False
2,21850866789130240,"Hello Twitter. In 2010, I almost died of the f...",en,fr,2011,1. Likely ILI infection,0,fr-en,False
3,21850866789130240,Ciao Twitter. Nel 2010 ho rischiato di morire ...,it,fr,2011,1. Likely ILI infection,0,fr-it,False
4,21850866789130240,"Bonjour Twitter. En 2010, j'ai frôlé la mort p...",fr,fr,2011,1. Likely ILI infection,0,fr-fr,True
...,...,...,...,...,...,...,...,...,...
21070,1631134812764291078,@user @user @user On veut gérer le Covid comme...,fr,fr,2023,3. Not Related to ILI or COVID-19 Infection,0,fr-fr,True
21071,1631134812764291078,@user @user @user Vogliamo gestire la Covid co...,it,fr,2023,3. Not Related to ILI or COVID-19 Infection,0,fr-it,False
21072,1631134812764291078,@usuario @usuario @usuario Queremos gestionar ...,es,fr,2023,3. Not Related to ILI or COVID-19 Infection,0,fr-es,False
21073,1631134812764291078,@user @user @user Man will das Covid wie die G...,de,fr,2023,3. Not Related to ILI or COVID-19 Infection,0,fr-de,False


In [36]:
# # save the split index
print(split_path.joinpath(f'split_idx_{experiment_type}.json'))
# with open(split_path.joinpath(f'split_idx_{experiment_type}.json'), 'w') as f:
#     json.dump(onetomany_split_idx, f)

/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/split_idx_one_to_many.json


<a name="manytoone"></a>

## 4. Many to One
### Get split index for each split of each language

In [32]:
experiment_type = 'many_to_one'

In [35]:
manytoone_split_idx = {}
for lang, splits_dict in split_id.items():
    print(lang)
    sub_df = full_data[full_data['target_lang']==lang]
    train = sub_df[sub_df['id'].isin(train_ids)]
    valid = sub_df[sub_df['id'].isin(valid_ids)]
    test = sub_df[sub_df['id'].isin(test_ids)]
    # test_original = test[test['target_lang']==test['lang']]
    print(train.shape, valid.shape, test.shape)
    
    manytoone_lang_split_idx = {}
    manytoone_lang_split_idx['train_idx'] = train.index.tolist()         # train on monolingual original
    manytoone_lang_split_idx['valid_idx'] = valid.index.tolist()         # valid on monolingual original
    manytoone_lang_split_idx['test_idx'] = test.index.tolist()           # test on monolingual translated 
    manytoone_split_idx[lang] = manytoone_lang_split_idx
print([j.keys() for i,j in manytoone_split_idx.items()])

de
(2567, 7) (857, 7) (860, 7)
en
(2567, 7) (857, 7) (860, 7)
es
(2567, 7) (857, 7) (860, 7)
fr
(2567, 7) (857, 7) (860, 7)
it
(2567, 7) (857, 7) (860, 7)
[dict_keys(['train_idx', 'valid_idx', 'test_idx']), dict_keys(['train_idx', 'valid_idx', 'test_idx']), dict_keys(['train_idx', 'valid_idx', 'test_idx']), dict_keys(['train_idx', 'valid_idx', 'test_idx']), dict_keys(['train_idx', 'valid_idx', 'test_idx'])]


In [38]:
# check each index is uniquely placed in one split type of one language only
# test_set will have repeat here 
all_split_types = manytoone_split_idx.get('de').keys()
print(all_split_types)

for type_to_test in all_split_types:
    for i in range(21):
        n = np.random.randint(0, full_data.shape[0])
        test_df = pd.DataFrame.from_dict(manytoone_split_idx)
        test_df = test_df[test_df.index==type_to_test]
        test_df = test_df[test_df.map(lambda x: n in x)]
        if (test_df.notna().sum().sum()==1) or test_df.notna().sum().sum()==0:
            pass
        else:
            print(type_to_test)
            print(n)
            print('matched more than once')

dict_keys(['train_idx', 'valid_idx', 'test_idx'])


In [36]:
# # save the split index
print(split_path.joinpath(f'split_idx_{experiment_type}.json'))
# with open(split_path.joinpath(f'split_idx_{experiment_type}.json'), 'w') as f:
#     json.dump(manytoone_split_idx, f)

/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2
